## Vehicle Classification
So far, we have extracted the vehicles from the frame and stored them in the folder. Next, we will perform the classification of vehicles present in the video. The training data is divided into 3 classes:

1. 2/3 wheelers (motorbikes, rickshaws)
2. 4 wheelers (cars)
3. More-than-4 wheelers (buses, trucks, etc.)

We have to manually label the data, in the sense that, you can make 3 folders for each of these 3 classes. And then, manually put the cropped images into one of the folders depending on the class.  Now your data is ready for training.
Once done you can proceed forward with this notebook and train an algorithm to classify the vehicles into above mentioned three classes.

In [3]:
import matplotlib
%matplotlib inline

import numpy as np
from PIL import Image
import random

There are basically two ways how you can feed the data during training.
- The first approach which is used here is to load all the images at once in the form of .npz file (zipped numpy array). Note that, the images are read as a numpy array. This can work if the memory used by the data is relatively less. Here, the size of the images are (50,50) which is a small size and the total dataset is 12k. So the dataset is relatively small.
- The second approach is to load the dataset during training runtime using ImageDataGenerator function of Keras. Here, only a batch of data gets loaded into the memory.

Here since dataset is small, we will take the first approach of storing files as npz format.

In [1]:

video = "Vehicles" #file name of npz
mode = "test"
file_name = "demo.npz"
to_stack = False

# Let's define a dictionary that maps the different classes we have to numbers
dict = {'twoW':0, 'threeFourW':1, 'fivePlusW':2}
for vehicle in sorted(dict):
	
	count = 0
	arr_stack = np.zeros((1, 50, 50, 3))
	y = dict[vehicle] * np.ones((1, 1))
	s = np.array([dict[vehicle]])
	s = s[np.newaxis, :]


	for i in range(100):
		path = "images/"
		img_path = path + str(i) + ".jpg"	
		try:	
			img = Image.open(img_path)
			if count == 0:
				arr = np.array(img)
				arr_stack[0, :, :, :] = arr
				count = 1
			else: 
				arr = np.array(img)
				arr_stack = np.concatenate((arr_stack, arr[np.newaxis, ...]), axis = 0)
				y = np.concatenate((y, s), axis = 0)
				# print arr_stack.shape, s
		except IOError:
			print ("Error")

	if to_stack:
		arr_test = np.load(file_name)
		arr_stack = np.concatenate((arr_stack, arr_test['a']), axis = 0)
		y = np.concatenate((y, arr_test['b']), axis = 0)

	np.savez_compressed(video, a = arr_stack, b = y)
	to_stack = True
arr_test = np.load(file_name)
# print 'ff', arr_test['a'].shape, arr_test['b'].shape

In [ ]:
# Importing libraries to train the model
from __future__ import print_function
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
#from keras.optimizers import SGD
from keras.utils import np_utils
import numpy
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint

#### Defining variables and dataset for training

In [ ]:
batch_size = 32
nb_classes = 3
nb_epoch = 300
data_augmentation = True

# We have resized images as 50, 50 when storing
img_rows, img_cols = 50, 50
img_channels = 3

# We manually split data into train and test npz files and load them.
arr = numpy.load("trainVehicles.npz")
X_train = 255 - arr['a']
y_train = arr['b']
Y_train = np_utils.to_categorical(y_train, nb_classes)

arr = numpy.load("testVehicles.npz")
X_test = 255 - arr['a']
y_test = arr['b']
Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
X_train -= 0.5
X_test -= 0.5

print(X_train.shape, X_test.shape)


## Building and Training the model

In [ ]:
'''
Training Code
Save model after compiling it, in case you have to press ^C during the execution of fit function.
'''

model = Sequential()

model.add(Convolution2D(32, 3, 3, border_mode='same',
                        input_shape=(img_rows, img_cols, img_channels)))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Convolution2D(128, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(128, 3, 3, border_mode='same'))
model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.75))

model.add(Convolution2D(128, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(128, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.75))

model.add(Flatten())
model.add(Dense(512, W_regularizer=l2(0.1)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, W_regularizer=l2(0.1)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

#HERE: Save model after compiling it, in case you have to press ^C during the execution of fit function.
model_json = model.to_json()
with open("modelVehicles.json", "w") as json_file:
        json_file.write(model_json)

filename = "modelVehicles.h5"
checkpoint = ModelCheckpoint(filename, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callback_list = [checkpoint]

model.fit(X_train, Y_train,
          batch_size=batch_size,
	  callbacks=callback_list,
          nb_epoch=nb_epoch,
          validation_data=(X_test, Y_test),
          shuffle=True)

In [ ]:
'''
Code to check accuracies of trained models on any dataset.
'''
from __future__ import print_function
from keras.models import model_from_json
import numpy

arr = numpy.load("trainAundh.npz")
X_train = 255 - arr['a']
y_train = arr['b']

#Load the dataset for which you want to get the accuracy.
arr = numpy.load("testAundh.npz")
X_test = 255 - arr['a']
y_test = arr['b']
X_display = numpy.copy(X_test)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
X_train -= 0.5
X_test -= 0.5

path = '../data/'
model_name = 'modelVehicles.json'

json_file = open(model_name, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.compile(loss='categorical_crossentropy',
	      optimizer='adam',
	      metrics=['accuracy'])
print("CNN Loaded")
loaded_model.summary()
#Iterate over all the weights, run a forward pass, check accuracy
for i in range(1, 6):
	weights_name = 'modelVehicles.h5'
	loaded_model.load_weights(weights_name)
	all_predictions = loaded_model.predict(X_test, batch_size = 32, verbose = 0) #Gives class probabilities
	all_predictions = numpy.argmax(all_predictions, axis = 1) #Finds max probability. That is the output class of the image.
	all_predictions = all_predictions[:, numpy.newaxis] #Reshape to y_test.shape
	error = all_predictions == y_test #Find correctly classified images
	acc = float(numpy.sum(error))
	acc /= y_test.shape[0]
	acc *= 100
	print('accuracy', acc)
	


In [ ]:
# Display Image and its class, optionally display accuracy.

from __future__ import print_function
from keras.models import model_from_json

import numpy
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
from PIL import Image

def decode_predictions(predictions, images):
	
	# Function to display image and its class.
		
	dict_map = {0:'twoW', 1:'threeFourW', 2:'fivePlusW'}
	nImages = predictions.shape[0]
	print (nImages)
	for i in range(nImages):
		print(dict_map[predictions[i]] + " detected")
		plt.imshow(Image.fromarray(images[i]))
		plt.show()

arr = numpy.load("trainAundh.npz")
X_train = 255 - arr['a']
y_train = arr['b']

#Change the file name if you want to load any other dataset.
arr = numpy.load("testAundh.npz")
X_test = 255 - arr['a']
X_display = arr['a']
print('X_test', X_test.shape)
y_test = arr['b']
print('y_test', y_test.shape)


X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
X_train -= 0.5
X_test -= 0.5

print(X_test.shape)
print(X_train.shape)

# First, you load the model architecture which is present in the form of 'json' file. Then you load the trained model weights(.h5 file) into model architecture. 
json_file = open('modelVehicles.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("modelVehicles.h5")
print("CNN Loaded")

loaded_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


rng = numpy.arange(1988, 1998)
# Predicting on test dataset
X_toPredict = X_test[rng, :, :, :]

predictions = loaded_model.predict(X_toPredict, batch_size = 32, verbose = 0)
predictions = numpy.argmax(predictions, axis = 1)
print('predictions', predictions)
print(predictions[:, numpy.newaxis].shape, y_test.shape)
decode_predictions(predictions, X_display[rng, :, :, :])


# Checks the accuracy on the curent test dataset, by using the method described in get_accuracy.py
all_predictions = loaded_model.predict(X_test, batch_size = 32, verbose = 0)
all_predictions = numpy.argmax(all_predictions, axis = 1)
all_predictions = all_predictions[:, numpy.newaxis]
error = all_predictions == y_test
acc = float(numpy.sum(error))
acc /= y_test.shape[0]
acc *= 100
print('accuracy', acc)

